# Mapeamento de patentes

### Bibliotecas

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os, requests, zipfile

### Download das revistas

Fonte de dados: http://revistas.inpi.gov.br/rpi/#

In [2]:
url2 = 2557
while url2 <= 2660:
    zip_url = 'http://revistas.inpi.gov.br/txt/P' + str(url2) + '.zip'
    # Nome do arquivo na pasta
    file_name = 'P' + str(url2) + '.zip'

    # Pasta onde você deseja salvar o arquivo ZIP
    output_folder = 'data_proj_map_patentes'

    # Caminho completo para o arquivo ZIP
    zip_path = os.path.join(output_folder, file_name)

    # Verifique se a pasta de saída existe e a crie se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Faça o download do arquivo ZIP diretamente na pasta de saída
    response = requests.get(zip_url)

    if response.status_code == 200:
        with open(zip_path, "wb") as zip_file:
            zip_file.write(response.content)
        
        print(f"Arquivo ZIP salvo em '{zip_path}'")

        # Abra o arquivo ZIP e extraia o arquivo .xml que começa com "patente"
        with zipfile.ZipFile(zip_path) as zipf:
            for filename in zipf.namelist():
                if filename.startswith("Patente_"):
                    # Extraia o arquivo correspondente
                    with zipf.open(filename) as xml_file:
                        xml_content = xml_file.read()
                        xml_filename = os.path.join(output_folder, filename)
                        with open(xml_filename, "wb") as output_file:
                            output_file.write(xml_content)
                    
                    print(f"Arquivo XML {filename} extraído com sucesso em '{xml_filename}'")
                    break  # Pare a iteração após encontrar o primeiro arquivo

        # Excluir o arquivo ZIP após a extração, se necessário
        os.remove(zip_path)

    else:
        print(f"O download do arquivo ZIP falhou com o código de status {response.status_code}")
    url2 += 1

Arquivo ZIP salvo em 'data_proj_map_patentes\P2557.zip'
Arquivo XML Patente_2557_07012020.xml extraído com sucesso em 'data_proj_map_patentes\Patente_2557_07012020.xml'
Arquivo ZIP salvo em 'data_proj_map_patentes\P2558.zip'
Arquivo XML Patente_2558_14012020.xml extraído com sucesso em 'data_proj_map_patentes\Patente_2558_14012020.xml'
Arquivo ZIP salvo em 'data_proj_map_patentes\P2559.zip'
Arquivo XML Patente_2559_21012020.xml extraído com sucesso em 'data_proj_map_patentes\Patente_2559_21012020.xml'
Arquivo ZIP salvo em 'data_proj_map_patentes\P2560.zip'
Arquivo XML Patente_2560_28012020.xml extraído com sucesso em 'data_proj_map_patentes\Patente_2560_28012020.xml'
Arquivo ZIP salvo em 'data_proj_map_patentes\P2561.zip'
Arquivo XML Patente_2561_04022020.xml extraído com sucesso em 'data_proj_map_patentes\Patente_2561_04022020.xml'
Arquivo ZIP salvo em 'data_proj_map_patentes\P2562.zip'
Arquivo XML Patente_2562_11022020.xml extraído com sucesso em 'data_proj_map_patentes\Patente_2562_

### Raspagem dos dados

In [3]:
# Definir função para raspagem de um arquivo xml
def raspagem(soup):
    # Inicializar listas para armazenar os dados
    data_publicacao = []
    diretoria = []
    numero_revista = []
    codigo = []
    titulo = []
    numero = []
    data_deposito = []
    data_prioridade = []
    cod_prioridade = []
    classificacoes_internacionais = []
    classificacoes_nacionais = []
    titulo_inid = []
    titular_nome = []
    titular_uf = []
    titular_pais = []
    inventor_nome = []

    # Extrair os dados do xml
    for despacho in soup.find_all(lambda tag: tag.name == 'despacho' and '<codigo>3.1</codigo>' in str(tag)):
        data_publ = soup.revista['dataPublicacao']
        diret = soup.revista['diretoria']
        num_rev = soup.revista['numero']
        cod = despacho.find('codigo').text
        tit = despacho.find('titulo').text
        num = despacho.find('numero').text
        data = despacho.find('data-deposito').text
        data_prio = despacho.find('data-prioridade', inid="31").text if despacho.find('data-prioridade', inid="31") else None
        cod_prio1 = despacho.find('sigla-pais', inid="33").text if despacho.find('sigla-pais', inid="33") else None
        cod_prio2 = despacho.find('numero-prioridade', inid="32").text if despacho.find('numero-prioridade', inid="32") else None
        cod_prio = cod_prio1 + cod_prio2 if (cod_prio1 != None and cod_prio2 != None) else None
        internacionais = '; '.join(ci.text for ci in despacho.find_all('classificacao-internacional'))
        nacionais = '; '.join(cn.text for cn in despacho.find_all('classificacao-nacional'))
        tit_inid = despacho.find('titulo', inid="54").text if despacho.find('titulo', inid="54") else None
        titu_nome = despacho.find('nome-completo').text if despacho.find('nome-completo') else None
        titu_uf = despacho.find('uf').text if despacho.find('uf') else None
        titu_pais = despacho.find('sigla').text if despacho.find('sigla') else None
        invent_nome = despacho.find('inventor').text if despacho.find('inventor') else None
        # Incluir os dados na lista
        data_publicacao.append(data_publ)
        diretoria.append(diret)
        numero_revista.append(num_rev)
        codigo.append(cod)
        titulo.append(tit)
        numero.append(num)
        data_deposito.append(data)
        data_prioridade.append(data_prio)
        cod_prioridade.append(cod_prio)
        classificacoes_internacionais.append(internacionais)
        classificacoes_nacionais.append(nacionais)
        titulo_inid.append(tit_inid)
        titular_nome.append(titu_nome)
        titular_uf.append(titu_uf)
        titular_pais.append(titu_pais)
        inventor_nome.append(invent_nome)
        # Transformar listas em dataframe
        df = pd.DataFrame({
            'data_publicacao' : data_publicacao,
            'diretoria' : diretoria,
            'numero_revista' : numero_revista,
            'codigo' : codigo,
            'titulo' : titulo,
            'numero' : numero,
            'data_deposito' : data_deposito,
            'data_prioridade' : data_prioridade,
            'cod_prioridade' : cod_prioridade,
            'classif_internacion' : classificacoes_nacionais,
            'classif_nacion' : classificacoes_nacionais,
            'titulo_inid' : titulo_inid,
            'titular_nome' : titular_nome,
            'titular_uf' : titular_uf,
            'titular_pais' : titular_pais,
            'inventor_nome' : inventor_nome
            })
    return(df)

### Salvar dataframe completo

In [5]:
# Inicializar o dataframe
data = pd.DataFrame()
for file in os.listdir('data_proj_map_patentes'):
    filepath = os.path.join('data_proj_map_patentes', file)
    with open(filepath, encoding='utf-8') as arquivo_xml:
        conteudo = arquivo_xml.read()
    soup = BeautifulSoup(conteudo, 'xml')
    df = raspagem(soup)
    data = pd.concat([data, df], axis=0)

data.to_csv('data.csv', index=False)

### Verificar informações do dataframe completo

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14098 entries, 0 to 77
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   data_publicacao      14098 non-null  object
 1   diretoria            14098 non-null  object
 2   numero_revista       14098 non-null  object
 3   codigo               14098 non-null  object
 4   titulo               14098 non-null  object
 5   numero               14098 non-null  object
 6   data_deposito        14098 non-null  object
 7   data_prioridade      3031 non-null   object
 8   cod_prioridade       3031 non-null   object
 9   classif_internacion  14098 non-null  object
 10  classif_nacion       14098 non-null  object
 11  titulo_inid          14098 non-null  object
 12  titular_nome         14098 non-null  object
 13  titular_uf           10921 non-null  object
 14  titular_pais         14098 non-null  object
 15  inventor_nome        14098 non-null  object
dtypes: object(16

In [8]:
df=pd.read_csv('data.csv')

In [10]:
df.tail()

,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,classif_internacion,classif_nacion,titulo_inid,titular_nome,titular_uf,titular_pais,inventor_nome
14093,28/12/2021,Patente,2660,3.1,Publicação do Pedido de Patente ou de Certific...,BR 20 2020 012571-6,19/06/2020,NaN,NaN,B30B 15/14,B30B 15/14,SISTEMA COPIADOR PARA MÁQUINA DE PREPARAR E OU...,HUGO PHILIPPE,SC,BR,\nHUGO PHILIPPE\n
14094,28/12/2021,Patente,2660,3.1,Publicação do Pedido de Patente ou de Certific...,BR 20 2020 012581-3,19/06/2020,NaN,NaN,B62B 1/22; B62B 3/006,B62B 1/22; B62B 3/006,CONFIGURAÇÃO APLICADA EM CARRINHO PASSA FIO,MOACYR FOPPA JUNIOR,SC,BR,\nMOACYR FOPPA JUNIOR\n
14095,28/12/2021,Patente,2660,3.1,Publicação do Pedido de Patente ou de Certific...,BR 20 2020 012618-6,19/06/2020,NaN,NaN,A01D 34/86,A01D 34/86,APERFEIÇOAMENTO INTRODUZIDO EM SISTEMA ELETRÔNICO,MAGMAEL FERNANDES DOS SANTOS,PR,BR,\nMAGMAEL FERNANDES DOS SANTOS\n
14096,28/12/2021,Patente,2660,3.1,Publicação do Pedido de Patente ou de Certific...,BR 20 2020 012662-3,20/06/2020,NaN,NaN,A61L 2/10; C12H 1/16,A61L 2/10; C12H 1/16,DISPOSIÇÃO CONSTRUTIVA APLICADA EM CAIXA DE DE...,HUGO ABRAHÃO DE LIMA,RJ,BR,\nHUGO ABRAHÃO DE LIMA\n
14097,28/12/2021,Patente,2660,3.1,Publicação do Pedido de Patente ou de Certific...,BR 20 2020 026189-0,21/12/2020,15/06/2020,CN202021116029.3,NaN,NaN,MÁSCARA,"SHANDONG SHENGQUAN NEW MATERIALS CO., LTD.",NaN,CN,"\nTANG, DIYUAN\n"
